# Проект по SQL

**Задания**  
- Посчитать, сколько книг вышло после 1 января 2000 года;  
- Для каждой книги посчитать количество обзоров и среднюю оценку;  
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;  
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;  
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Ознакомление с данными

### Выведем books

In [3]:
query = '''SELECT * FROM books LIMIT 5'''
books = pd.io.sql.read_sql(sql=text(query), con = con)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Выведем authors

In [4]:
query = '''SELECT * FROM authors LIMIT 5'''
authors = pd.io.sql.read_sql(sql=text(query), con = con)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Выведем ratings

In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''
ratings = pd.io.sql.read_sql(sql=text(query), con = con)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Выведем reviews

In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''
reviews = pd.io.sql.read_sql(sql=text(query), con = con)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Выведем publishers

In [7]:
query = '''SELECT * FROM publishers LIMIT 5'''
publishers = pd.io.sql.read_sql(sql=text(query), con = con)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Выполнение заданий

Создадим функцию для выполнения запросов

In [8]:
def sql_query(query):
    data = pd.io.sql.read_sql(sql=text(query), con = con)
    return data

### Задача № 1

Необходимо посчитать сколько книг вышло после 1 января 2000 года

In [9]:
query = '''
SELECT COUNT(DISTINCT book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
books_count = sql_query(query)
books_count

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Задача № 2

Необходимо посчитать для каждой книги количество обзоров и среднюю оценку

In [10]:
query = '''
SELECT b.book_id AS book_id,
       b.title AS book_title,
       ROUND(AVG(rt.rating), 1) AS rating,
       COUNT(DISTINCT rw.review_id) AS cnt_review
FROM books AS b
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
LEFT JOIN reviews AS rw ON rt.book_id = rw.book_id
GROUP BY b.book_id
ORDER BY cnt_review DESC, rating DESC
'''
count_avg_books = sql_query(query)
count_avg_books

,book_id,book_title,rating,cnt_review
0,948,Twilight (Twilight #1),3.7,7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.4,6
2,656,The Book Thief,4.3,6
3,299,Harry Potter and the Chamber of Secrets (Harry...,4.3,6
4,734,The Glass Castle,4.2,6
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,4.0,0
996,387,Leonardo's Notebooks,4.0,0
997,221,Essential Tales and Poems,4.0,0
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.7,0


Книга Twilight (Twilight #1) имеет больше всего обзоров (7) и рейтинг 3.7

### Задача № 3

Необходимо определить издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query = '''
SELECT p.publisher AS publisher,
       COUNT(b.book_id) AS cnt_books
FROM books AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY publisher
ORDER BY cnt_books DESC
LIMIT 1
'''
publisher_top1 = sql_query(query)
publisher_top1

,publisher,cnt_books
0,Penguin Books,42


Издательство 'Penguin Books' выпустило наибольшее количество книг толще 50 страниц - 42 шт.

### Задача № 4

Необходимо определить автора с самой высокой средней оценкой книг. Учитываются только книги с 50 и более оценками

In [39]:
query = '''
SELECT a.author AS author,
       ROUND(AVG(r.rating), 2) AS rating,
       COUNT(r.rating) AS cnt_rating
FROM books AS b
JOIN ratings AS r ON b.book_id = r.book_id
JOIN authors AS a ON b.author_id = a.author_id
WHERE b.book_id IN (
       SELECT book_id
       FROM ratings
       GROUP BY book_id
       HAVING COUNT(DISTINCT rating_id) >= 50)
GROUP BY a.author
ORDER BY rating DESC
LIMIT 1
'''
author_top1 = sql_query(query)
author_top1

,author,rating,cnt_rating
0,J.K. Rowling/Mary GrandPré,4.29,310


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré	 (4.29)

### Задача № 5

Необходимо посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query = '''
SELECT AVG(cnt_review) AS avg_review
FROM (SELECT COUNT(review_id) AS cnt_review
    FROM reviews
    WHERE username IN (
        SELECT rt.username
        FROM ratings AS rt
        GROUP BY rt.username
        HAVING COUNT(DISTINCT rt.rating_id) > 48)
GROUP BY username) AS avg
'''
avg_reviews = sql_query(query)
avg_reviews

,avg_review
0,24.0


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок равняется 24.

## Вывод

1. На входе были получены следующие таблицы:  
- books  
- authors  
- ratings  
- reviews  
- publishers  

2. На этапе подготовки мы импортировали библиотеки и подключили базу данных.  


3. Произвели ознакомление со всеми таблицами из списка.  


4. Далее мы выявили следующую информацию:  
- после 1 января 2000 года вышло 819 книг  
- для каждой книги определили средний рейтинг и количество обзоров на неё. Книга Twilight (Twilight #1) имеет больше всего обзоров (7) и рейтинг 3.7  
- издательство 'Penguin Books' выпустило наибольшее количество книг толще 50 страниц - 42 шт.  
- автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré (4.29)  
- среднее количество обзоров от пользователей, которые поставили больше 48 оценок равняется 24.